In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import concatenate
from keras.optimizers import SGD
from keras.models import load_model
from keras import regularizers

D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
df = pd.read_csv('./data.csv')
train_df = df[pd.to_datetime(df['quarter'])<="2019-01-01"]
test_df = df[pd.to_datetime(df['quarter'])=="2019-04-01"]

In [3]:
train_features=train_df.copy()
test_features=test_df.copy()
train_mret=train_features[['mret_net_weighted_qt_L1','mret_net_weighted_qt_L2','mret_net_weighted_qt_L3','mret_net_weighted_qt_L4']]
test_mret=test_features[['mret_net_weighted_qt_L1','mret_net_weighted_qt_L2','mret_net_weighted_qt_L3','mret_net_weighted_qt_L4']]
train_labels = train_features.pop('weight')
test_labels = test_features.pop('weight')
train_features.drop(['quarter','mret_net_weighted_qt_L1','mret_net_weighted_qt_L2',
                     'mret_net_weighted_qt_L3','mret_net_weighted_qt_L4'],axis=1,inplace=True)
test_features.drop(['quarter','mret_net_weighted_qt_L1','mret_net_weighted_qt_L2',
                    'mret_net_weighted_qt_L3','mret_net_weighted_qt_L4'],axis=1,inplace=True)

In [4]:
train_features.columns.values

array(['nav_latest_L1', 'size_q_L1', 'yield_L1', 'per_com_L1',
       'per_pref_L1', 'maturity_L1', 'exp_ratio_weighted_L1',
       'turn_ratio_weighted_L1', 'age_L1', 'manager_tenure_L1', 'ioc_1',
       'ioc_2', 'ioc_3', 'ioc_4', 'ioc_5', 'ioc_6', 'retail_fund_i',
       'inst_fund_i', 'dead_flag_L1_i', 'lipper_asset_cd_1',
       'lipper_asset_cd_2', 'lipper_asset_cd_3', 'prccq_L1', 'trt1q_L1',
       'prccd_sd_L1', 'prccd_sd_qtr_L1', 'annret_L1', 'salegrowth_L1',
       'size_L1', 'sizemkt_L1', 'bookleverage_L1', 'roa_L1',
       'cashtoasset_L1', 'debtst_L1', 'debtlt_L1', 'q_L1', 'Pnaive_L1',
       'buyback_L1', 'cshareissued_L1', 'netleverage_L1', 'capitalexp_L1',
       'cashfvol_L1', 'trt1q_L2', 'trt1q_L3', 'trt1q_L4'], dtype=object)

In [5]:
train_features=np.array(train_features)
test_features=np.array(test_features)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)
train_mret=np.array(train_mret)
test_mret=np.array(test_mret)

In [6]:
#Split Data for Test and Training
X_train = train_features
Xexog_train = train_mret
Y_train = train_labels

X_test = test_features
Xexog_test = test_mret
Y_test = test_labels

#Scale the predictors for training
Xscaler_train =  MinMaxScaler(feature_range=(-1,1))
X_scaled_train = Xscaler_train.fit_transform(X_train)

Xexog_scaler_train =  MinMaxScaler(feature_range=(-1,1))
Xexog_scaled_train = Xexog_scaler_train.fit_transform(Xexog_train)

# Scale the data for testing using the in-sample transformation from earlier
X_scaled_test = Xscaler_train.transform(X_test)
Xexog_scaled_test = Xexog_scaler_train.transform(Xexog_test)


In [7]:
"""
This model fits a neural network for each group of macro variables and
ensembles those macro networks. It also adds the yields variables in the
last hidden layer.
"""

archi = [32, 16, 8]
dropout_u = 0.3
A = [6,10,7,8,13,13,6,6,10,10,1,1,1,1,1,1,1,1,1,1,1,1,2,5,2,2,2,9,5,5,3,7,2,11,3,2,2,2,4,3,9,9,11,12,2]

# Keras requires 3D tuples for training.

Xexog_scaled_train_expand = np.expand_dims(Xexog_scaled_train, axis=1)


Y_train_expand1 = np.expand_dims(Y_train, axis=1)
Y_train_expand = np.expand_dims(Y_train_expand1, axis=1)



Xexog_scaled_test_expand = np.expand_dims(Xexog_scaled_test, axis=1)

Y_test_expand1 = np.expand_dims(Y_test, axis=1)
Y_test_expand = np.expand_dims(Y_test_expand1, axis=1)

In [8]:
# Split X_train / X_test by group
X_scaled_train_grouped = []
X_scaled_test_grouped = []
n_groups = len(np.unique(A))
for i, group in enumerate(np.unique(A)):
    temp = X_scaled_train[:,[x==group for x in A]]
    X_scaled_train_grouped.append(np.expand_dims(temp, axis=1))
    temp = X_scaled_test[:,[x==group for x in A]]
    X_scaled_test_grouped.append(np.expand_dims(temp, axis=1))

# seed numpy and tf
tf.set_random_seed(123)
np.random.seed(123)


# Define Model Architecture
n = len(archi)
layers = dict()

In [9]:
# Model for macro variables
for i in range(n+1):
    if i == 0:
        layers['ins_main'] = [Input(shape=(1,X_scaled_train_grouped[j].shape[2])) for j in range(n_groups)]
    elif i == 1:
        layers['dropout'+str(i)] = [Dropout(dropout_u)(input_tensor) for input_tensor in layers['ins_main']]
        layers['hiddens'+str(i)] = [Dense(archi[i-1], activation='relu',kernel_initializer='he_normal',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.001))(input_tensor) for input_tensor in layers['dropout'+str(i)]]
    elif i > 1 & i <= n:
        layers['dropout'+str(i)] = [Dropout(dropout_u)(input_tensor) for input_tensor in layers['hiddens'+str(i-1)]]
        layers['hiddens'+str(i)] = [Dense(archi[i-1], activation='relu',kernel_initializer='he_normal',kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.001))(input_tensor) for input_tensor in layers['dropout'+str(i)]]

# Model for yiel variables
layers['ins_exog'] = Input(shape=(1,Xexog_scaled_train_expand.shape[2]))
# Merge macro group models
layers['merge'] = concatenate(layers['hiddens'+str(n)])
# Merge macro and yields models
layers['merge1'] = concatenate([layers['merge'], layers['ins_exog']])
layers['dropout_final'] = Dropout(dropout_u)(layers['merge1'])
layers['BN'] = BatchNormalization()(layers['dropout_final'])
layers['output'] = Dense(Y_train_expand.shape[2], kernel_initializer='he_normal')(layers['BN'])

model = Model(inputs=layers['ins_main']+[layers['ins_exog']],
              outputs=layers['output'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 12)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 9)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1, 3)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1, 1)         0                                            
____________________________________________________________________________________________

In [10]:
# Compile model
sgd_fine = SGD(lr=0.015, momentum=0.9, decay=0.01, nesterov=True)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=1e-6,
                              patience=20, verbose=0, mode='auto')

model.compile(loss='mean_squared_error', optimizer=sgd_fine)
history=model.fit(X_scaled_train_grouped+[Xexog_scaled_train_expand], Y_train_expand,
                  epochs=500,validation_split=0.15, batch_size=32, shuffle=True,verbose=1)

# Perform Forecast and Test Model
Ypred = model.predict(X_scaled_test_grouped+[Xexog_scaled_test_expand])
Ypred = np.squeeze(Ypred,axis=1)

score = model.evaluate(X_scaled_test_grouped+[Xexog_scaled_test_expand], Y_test_expand, verbose=1)
print("Test score:", score)



Train on 17203 samples, validate on 3036 samples
Epoch 1/500
17203/17203 [==============================] - 5s 282us/step - loss: 15.9799 - val_loss: 9.9620
Epoch 2/500
17203/17203 [==============================] - 3s 146us/step - loss: 8.3098 - val_loss: 6.9932
Epoch 3/500
17203/17203 [==============================] - 3s 157us/step - loss: 6.4191 - val_loss: 5.8176
Epoch 4/500
17203/17203 [==============================] - 3s 165us/step - loss: 5.5316 - val_loss: 5.1456
Epoch 5/500
17203/17203 [==============================] - 3s 155us/step - loss: 5.0247 - val_loss: 4.7794
Epoch 6/500
17203/17203 [==============================] - 3s 155us/step - loss: 4.6866 - val_loss: 4.4944
Epoch 7/500
17203/17203 [==============================] - 3s 154us/step - loss: 4.4582 - val_loss: 4.3042
Epoch 8/500
17203/17203 [==============================] - 3s 160us/step - loss: 4.2734 - val_loss: 4.1242
Epoch 9/500
17203/17203 [==============================] - 3s 156us/step - loss: 4.1274 - val

17203/17203 [==============================] - 3s 159us/step - loss: 2.6863 - val_loss: 2.6784
Epoch 75/500
17203/17203 [==============================] - 3s 157us/step - loss: 2.6857 - val_loss: 2.6663
Epoch 76/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.6698 - val_loss: 2.6648
Epoch 77/500
17203/17203 [==============================] - 3s 161us/step - loss: 2.6687 - val_loss: 2.6531
Epoch 78/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.6685 - val_loss: 2.6491
Epoch 79/500
17203/17203 [==============================] - 3s 156us/step - loss: 2.6562 - val_loss: 2.6396
Epoch 80/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.6574 - val_loss: 2.6343
Epoch 81/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.6471 - val_loss: 2.6462
Epoch 82/500
17203/17203 [==============================] - 3s 161us/step - loss: 2.6555 - val_loss: 2.6303
Epoch 83/500
17203/17203 [===============

17203/17203 [==============================] - 3s 159us/step - loss: 2.4454 - val_loss: 2.4371
Epoch 150/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.4318 - val_loss: 2.4332
Epoch 151/500
17203/17203 [==============================] - 3s 157us/step - loss: 2.4446 - val_loss: 2.4294
Epoch 152/500
17203/17203 [==============================] - 3s 157us/step - loss: 2.4458 - val_loss: 2.4306
Epoch 153/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.4388 - val_loss: 2.4313
Epoch 154/500
17203/17203 [==============================] - 3s 161us/step - loss: 2.4295 - val_loss: 2.4276
Epoch 155/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.4341 - val_loss: 2.4185
Epoch 156/500
17203/17203 [==============================] - 3s 157us/step - loss: 2.4246 - val_loss: 2.4185
Epoch 157/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.4326 - val_loss: 2.4174
Epoch 158/500
17203/17203 [======

17203/17203 [==============================] - 3s 161us/step - loss: 2.3287 - val_loss: 2.3178
Epoch 225/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.3358 - val_loss: 2.3093
Epoch 226/500
17203/17203 [==============================] - 3s 165us/step - loss: 2.3193 - val_loss: 2.3169
Epoch 227/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.3264 - val_loss: 2.3165
Epoch 228/500
17203/17203 [==============================] - 3s 171us/step - loss: 2.3215 - val_loss: 2.3122
Epoch 229/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.3247 - val_loss: 2.3136
Epoch 230/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.3247 - val_loss: 2.3074
Epoch 231/500
17203/17203 [==============================] - 3s 161us/step - loss: 2.3164 - val_loss: 2.3089
Epoch 232/500
17203/17203 [==============================] - 3s 166us/step - loss: 2.3204 - val_loss: 2.3038
Epoch 233/500
17203/17203 [======

17203/17203 [==============================] - 3s 160us/step - loss: 2.2519 - val_loss: 2.2453
Epoch 300/500
17203/17203 [==============================] - 3s 166us/step - loss: 2.2682 - val_loss: 2.2387
Epoch 301/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.2501 - val_loss: 2.2387
Epoch 302/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.2554 - val_loss: 2.2376
Epoch 303/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.2493 - val_loss: 2.2459
Epoch 304/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.2489 - val_loss: 2.2425
Epoch 305/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.2522 - val_loss: 2.2374
Epoch 306/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.2500 - val_loss: 2.2379
Epoch 307/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.2417 - val_loss: 2.2346
Epoch 308/500
17203/17203 [======

17203/17203 [==============================] - 3s 157us/step - loss: 2.1986 - val_loss: 2.1953
Epoch 375/500
17203/17203 [==============================] - 3s 161us/step - loss: 2.1997 - val_loss: 2.1941
Epoch 376/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.2111 - val_loss: 2.1904
Epoch 377/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.2038 - val_loss: 2.1955
Epoch 378/500
17203/17203 [==============================] - 3s 162us/step - loss: 2.2003 - val_loss: 2.1882
Epoch 379/500
17203/17203 [==============================] - 3s 165us/step - loss: 2.2066 - val_loss: 2.1909
Epoch 380/500
17203/17203 [==============================] - 3s 157us/step - loss: 2.2063 - val_loss: 2.1926
Epoch 381/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.1968 - val_loss: 2.1901
Epoch 382/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.1995 - val_loss: 2.1862
Epoch 383/500
17203/17203 [======

17203/17203 [==============================] - 3s 157us/step - loss: 2.1649 - val_loss: 2.1591
Epoch 450/500
17203/17203 [==============================] - 3s 158us/step - loss: 2.1717 - val_loss: 2.1536
Epoch 451/500
17203/17203 [==============================] - 3s 161us/step - loss: 2.1702 - val_loss: 2.1523
Epoch 452/500
17203/17203 [==============================] - 3s 161us/step - loss: 2.1635 - val_loss: 2.1583
Epoch 453/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.1650 - val_loss: 2.1503
Epoch 454/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.1703 - val_loss: 2.1512
Epoch 455/500
17203/17203 [==============================] - 3s 162us/step - loss: 2.1687 - val_loss: 2.1512
Epoch 456/500
17203/17203 [==============================] - 3s 160us/step - loss: 2.1708 - val_loss: 2.1553
Epoch 457/500
17203/17203 [==============================] - 3s 159us/step - loss: 2.1709 - val_loss: 2.1498
Epoch 458/500
17203/17203 [======